In [1]:
pip install optuna

  Obtaining dependency information for optuna from https://files.pythonhosted.org/packages/69/60/87a06ef66b34cbe2f2eb0ab66f003664404a7f40c21403a69fad7e28a82b/optuna-3.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for cmaes>=0.10.0 from https://files.pythonhosted.org/packages/f7/46/7d9544d453346f6c0c405916c95fdb653491ea2e9976cabb810ba2fe8cd4/cmaes-0.10.0-py3-none-any.whl.metadata
   --------------------------------------- 404.2/404.2 kB 12.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
def smape(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    result = np.mean(v) * 100
    return result

# XGB

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import math
import pandas as pd
import numpy as np
import datetime as dt
from xgboost import XGBRegressor
import optuna

def smape(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    result = np.mean(v) * 100
    return result

train = pd.read_csv('train_0812.csv', encoding='cp949')
train.solar_yes = train.solar_yes.astype(int)
train.ess_yes = train.ess_yes.astype(int)
train.sy_ey = train.sy_ey.astype(int)
train.sy_en = train.sy_en.astype(int)

# train.drop(['building_number','date_time','building_type'])

train['hour_sin'] = np.sin(2 * np.pi * train['hour']/23.0)
train['hour_cos'] = np.cos(2 * np.pi * train['hour']/23.0)

train['date_sin'] = -np.sin(2 * np.pi * (train['month']+train['day']/31)/12)
train['date_cos'] = -np.cos(2 * np.pi * (train['month']+train['day']/31)/12)

train['month_sin'] = -np.sin(2 * np.pi * train['month']/12.0)
train['month_cos'] = -np.cos(2 * np.pi * train['month']/12.0)
## cyclic transformation on weekday
train['weekday_sin'] = -np.sin(2 * np.pi * (train['weekday']+1)/7.0)
train['weekday_cos'] = -np.cos(2 * np.pi * (train['weekday']+1)/7.0)


train['work_time'] = 0
train['low_day'] = 0
train['particular'] = 0
trains = [train[train.building_number == num] for num in range(1,101,1)]
# aa = trains[9]
# aa['work_time']  = (aa['hour'].isin([6,7,8])).astype(int)
# aa.describe()

## cluster 0
for num in [10,13]:
    temp_df = trains[num-1]
    temp_df['work_time'] = (temp_df['hour'].isin([6,7,8])).astype(int)
    
    trains[num-1] = temp_df
for num in [14]:
    temp_df = trains[num-1]
    temp_df['work_time'] = (temp_df['hour'].isin([6,7])).astype(int)
    temp_df['particular'] = temp_df['hour'].isin([0,8,12,18]).astype(int)
    trains[num-1] = temp_df
for num in [15]:
    temp_df = trains[num-1]
    temp_df['work_time'] = (temp_df['hour'].isin([4,5,6,7,8])).astype(int)
    trains[num-1] = temp_df

## cluster 1
for num in [17,18,19,21,22,69,70,72]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=18)&(temp_df['hour']>=6)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [45,46,47,48,49,51,52,73]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=6)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [26,28,76,78,80,82,83,84]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=18)&(temp_df['hour']>=8)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [71]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=18)*(temp_df['hour']>=8)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    temp_df['particular'] = (((temp_df['weekday']==5)&(temp_df['hour'].isin([10,11,13,14])))
                               |((temp_df['weekday'].isin([0,1,2,3,4,5]))&(temp_df['hour'].isin([0,1,2,3,4])))).astype(int)
    trains[num-1] = temp_df
for num in [53]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=4)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [55]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=19)&(temp_df['hour']>=5)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [50]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=20)&(temp_df['hour']>=5)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [56]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=20)&(temp_df['hour']>=5)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    temp_df['particular'] = ((temp_df['weekday']==2)&(temp_df['hour']==16)).astype(int)
    trains[num-1] = temp_df
for num in [60]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=5)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [20,23]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=17)&(temp_df['hour']>=6)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [58,59,77]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=20)&(temp_df['hour']>=6)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [45,46,47,48,49,51,52,73]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=6)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [27,74,79]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=18)&(temp_df['hour']>=7)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [57]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=22)&(temp_df['hour']>=7)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [24,25]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=20)&(temp_df['hour']>=8)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [29,30]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=17)&(temp_df['hour']>=9)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [31]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=20)&(temp_df['hour']>=9)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    trains[num-1] = temp_df
for num in [50]:
    temp_df = trains[num-1]
    temp_df['particular'] = (((temp_df['weekday']==2)&(temp_df['hour']==16))|((temp_df['weekday']==2)&(temp_df['hour']==16))).astype(int)
    trains[num-1] = temp_df
for num in [1]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=7)).astype(int)
    temp_df['low_day'] = (temp_df['weekday']==6).astype(int)
    trains[num-1] = temp_df
for num in [8]:
    temp_df = trains[num-1]
    temp_df['low_day'] = (temp_df['weekday']==6).astype(int)
    trains[num-1] = temp_df
for num in [12]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=5)).astype(int)
    temp_df['low_day'] = (temp_df['weekday']==6).astype(int)
    trains[num-1] = temp_df
for num in [88,90]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=22)&(temp_df['hour']>=8)).astype(int)
    trains[num-1] = temp_df
for num in [86]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=22)&(temp_df['hour']>=8)).astype(int)
    temp_df['particular'] = ((temp_df['month']==6)&(temp_df['day']==26)).astype(int)
    trains[num-1] = temp_df
for num in [87,89,91]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=9)).astype(int)
    trains[num-1] = temp_df
for num in [4]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=18)&(temp_df['hour']>=8)).astype(int)
    trains[num-1] = temp_df
for num in [7,16]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=22)&(temp_df['hour']>=5)).astype(int)
    trains[num-1] = temp_df
for num in [9]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=23)&(temp_df['hour']>=7)).astype(int)
    trains[num-1] = temp_df
for num in [11]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=7)).astype(int)
    trains[num-1] = temp_df
for num in [36]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=20)&(temp_df['hour']>=9)).astype(int)
    trains[num-1] = temp_df
for num in [75]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=15)&(temp_df['hour']>=7)).astype(int)
    trains[num-1] = temp_df
for num in [92]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=22)&(temp_df['hour']>=6)).astype(int)
    trains[num-1] = temp_df

## cluster 2
for num in [32,33,34]:
    temp_df = trains[num-1]
    temp_df = temp_df[temp_df.month>=7]
    trains[num-1] = temp_df
for num in [61,65,66,67,68]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=7)&(temp_df['hour']<=23)).astype(int)
    trains[num-1] = temp_df
for num in [62,63,64]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=8)&(temp_df['hour']<=23)).astype(int)
    trains[num-1] = temp_df

## cluster 3
for num in [93,94,96,97,98,100]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=6)&(temp_df['hour']<=22)).astype(int)
    trains[num-1] = temp_df
for num in [42,43,44]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=9)&(temp_df['hour']<=20)).astype(int)
    trains[num-1] = temp_df
for num in [2,85,99]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=8)&(temp_df['hour']<=21)).astype(int)
    trains[num-1] = temp_df
for num in [38,40,95]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=8)&(temp_df['hour']<=20)).astype(int)
    trains[num-1] = temp_df
for num in [3,81]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=9)&(temp_df['hour']<=21)).astype(int)
    trains[num-1] = temp_df
for num in [6,41]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=8)&(temp_df['hour']<=22)).astype(int)
    trains[num-1] = temp_df
for num in [37,39]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=7)&(temp_df['hour']<=21)).astype(int)
    trains[num-1] = temp_df
for num in [35]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=7)&(temp_df['hour']<=23)).astype(int)
    trains[num-1] = temp_df
for num in [54]:
    temp_df = trains[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=5)&(temp_df['hour']<=20)).astype(int)
    temp_df['low_day'] = (temp_df['weekday']==0).astype(int)
    trains[num-1] = temp_df
for num in [5]:
    temp_df = trains[num-1]
    temp_df['particular'] = (((temp_df['weekday']==4)&(temp_df['hour'].isin([11,12,13,14,15,16,17,18,19,20])))|
                              ((temp_df['weekday']==5)&(temp_df['hour'].isin([11,12,13,14,15,16,17,18,19,20])))|
                              ((temp_df['weekday']==6)&(temp_df['hour'].isin([8,9,10,11,12,13,14,15,16,17,18])))).astype(int)
    trains[num-1] = temp_df
    

df = pd.concat(trains, ignore_index=True)
df = df.drop(['date_time','hour','day','month','weekday','date'], axis=1)
df.building_type = df.building_type.astype('category')



df_0 = df[df.km_cluster==0]
df_1 = df[df.km_cluster==1]
df_2 = df[df.km_cluster==2]
df_3 = df[df.km_cluster==3]

df_0 = pd.get_dummies(df_0)
df_1 = pd.get_dummies(df_1)
df_2 = pd.get_dummies(df_2)
df_3 = pd.get_dummies(df_3)


X_0 = df_0.drop('power_consumption', axis=1)
y_0 = df_0['power_consumption']
y_0_log = np.log(y_0)
X_0_train = X_0.drop('km_cluster', axis=1)

X_1 = df_1.drop('power_consumption', axis=1)
y_1 = df_1['power_consumption']
y_1_log = np.log(y_1)
X_1_train = X_1.drop('km_cluster', axis=1)

X_2 = df_2.drop('power_consumption', axis=1)
y_2 = df_2['power_consumption']
y_2_log = np.log(y_2)
X_2_train = X_2.drop('km_cluster', axis=1)

X_3 = df_3.drop('power_consumption', axis=1)
y_3 = df_3['power_consumption']
y_3_log = np.log(y_3)
X_3_train = X_3.drop('km_cluster', axis=1)


# Standard Scaling for continuous data
con_li = ['hour_sin', 'hour_cos', 'date_sin', 'date_cos', 'month_sin', 'month_cos', 'weekday_sin', 'weekday_cos', 'temperature', 'windspeed', 
'humidity', 'total_area', 'cooling_area', 'solar_power_capacity', 'ess_capacity', 'pcs_capacity', 'discomfort', 'temperature_3', 
'discomfort_3', 'ma_dis_3', 'ma_dis_5', 'CDH', 'temperature_mean', 'discomfort_mean', 'CDH_mean', 'sensory_temperature']

# ## one-hot encoding for categorical data
cat_li = ['building_number','holiday','air_yes','solar_yes',
         'ess_yes','sy_ey','sy_en','low_day','particular']


for col in con_li :
    if X_0_train[col].std() == 0:
        X_0_train = X_0_train.drop(col, axis=1)
    else :
        X_0_train[col] = (X_0_train[col] - X_0_train[col].mean())/X_0_train[col].std()
        
for col in cat_li :
    if X_0_train[col].nunique() == 1:        
        X_0_train_train = X_0_train.drop(col,axis=1)

for col in con_li :
    if X_1_train[col].std() == 0:
        X_1_train = X_1_train.drop(col, axis=1)
    else :
        X_1_train[col] = (X_1_train[col] - X_1_train[col].mean())/X_1_train[col].std()
        
for col in cat_li :
    if X_1_train[col].nunique() == 1:
        X_1_train = X_1_train.drop(col,axis=1)


for col in con_li :
    if X_2_train[col].std() == 0:
        X_2_train = X_2_train.drop(col, axis=1)
    else :
        X_2_train[col] = (X_2_train[col] - X_2_train[col].mean())/X_2_train[col].std()
        
for col in cat_li :
    if X_2_train[col].nunique() == 1:
        X_2_train = X_2_train.drop(col,axis=1)

for col in con_li :
    if X_3_train[col].std() == 0:
        X_3_train = X_3_train.drop(col, axis=1)
    else :
        X_3_train[col] = (X_3_train[col] - X_3_train[col].mean())/X_3_train[col].std()
        
for col in cat_li :
    if X_3_train[col].nunique() == 1:
        X_3_train = X_3_train.drop(col,axis=1)

train = pd.read_csv('train_0812.csv', encoding='cp949')
test = pd.read_csv('test_0812.csv', encoding='cp949')
cluster_dict = train.groupby('building_number')['km_cluster'].first().to_dict()
test['km_cluster'] = test['building_number'].map(cluster_dict)

test.solar_yes = test.solar_yes.astype(int)
test.ess_yes = test.ess_yes.astype(int)
test.sy_ey = test.sy_ey.astype(int)
test.sy_en = test.sy_en.astype(int)

# test.drop(['building_number','date_time','building_type'])

test['hour_sin'] = np.sin(2 * np.pi * test['hour']/23.0)
test['hour_cos'] = np.cos(2 * np.pi * test['hour']/23.0)

test['date_sin'] = -np.sin(2 * np.pi * (test['month']+test['day']/31)/12)
test['date_cos'] = -np.cos(2 * np.pi * (test['month']+test['day']/31)/12)

test['month_sin'] = -np.sin(2 * np.pi * test['month']/12.0)
test['month_cos'] = -np.cos(2 * np.pi * test['month']/12.0)
## cyclic transformation on weekday
test['weekday_sin'] = -np.sin(2 * np.pi * (test['weekday']+1)/7.0)
test['weekday_cos'] = -np.cos(2 * np.pi * (test['weekday']+1)/7.0)


test['work_time'] = 0
test['low_day'] = 0
test['particular'] = 0
tests = [test[test.building_number == num] for num in range(1,101,1)]
# aa = tests[9]
# aa['work_time']  = (aa['hour'].isin([6,7,8])).astype(int)
# aa.describe()


C:\Users\yeji4\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\yeji4\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\yeji4\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
C:\Users\yeji4\AppData\Local\Temp/ipykernel_12252/2408098666.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['work_time'] = (temp_df[

In [3]:

## cluster 0
for num in [10,13]:
    temp_df = tests[num-1]
    temp_df['work_time'] = (temp_df['hour'].isin([6,7,8])).astype(int)
    
    tests[num-1] = temp_df
for num in [14]:
    temp_df = tests[num-1]
    temp_df['work_time'] = (temp_df['hour'].isin([6,7])).astype(int)
    temp_df['particular'] = temp_df['hour'].isin([0,8,12,18]).astype(int)
    tests[num-1] = temp_df
for num in [15]:
    temp_df = tests[num-1]
    temp_df['work_time'] = (temp_df['hour'].isin([4,5,6,7,8])).astype(int)
    tests[num-1] = temp_df

## cluster 1
for num in [17,18,19,21,22,69,70,72]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=18)&(temp_df['hour']>=6)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [45,46,47,48,49,51,52,73]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=6)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [26,28,76,78,80,82,83,84]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=18)&(temp_df['hour']>=8)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [71]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=18)*(temp_df['hour']>=8)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    temp_df['particular'] = (((temp_df['weekday']==5)&(temp_df['hour'].isin([10,11,13,14])))
                               |((temp_df['weekday'].isin([0,1,2,3,4,5]))&(temp_df['hour'].isin([0,1,2,3,4])))).astype(int)
    tests[num-1] = temp_df
for num in [53]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=4)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [55]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=19)&(temp_df['hour']>=5)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [50]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=20)&(temp_df['hour']>=5)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [56]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=20)&(temp_df['hour']>=5)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    temp_df['particular'] = ((temp_df['weekday']==2)&(temp_df['hour']==16)).astype(int)
    tests[num-1] = temp_df
for num in [60]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=5)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [20,23]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=17)&(temp_df['hour']>=6)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [58,59,77]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=20)&(temp_df['hour']>=6)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [45,46,47,48,49,51,52,73]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=6)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [27,74,79]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=18)&(temp_df['hour']>=7)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [57]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=22)&(temp_df['hour']>=7)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [24,25]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=20)&(temp_df['hour']>=8)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [29,30]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=17)&(temp_df['hour']>=9)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [31]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=20)&(temp_df['hour']>=9)).astype(int)
    temp_df['low_day'] = (temp_df['weekday'].isin([5,6])).astype(int)
    tests[num-1] = temp_df
for num in [50]:
    temp_df = tests[num-1]
    temp_df['particular'] = (((temp_df['weekday']==2)&(temp_df['hour']==16))|((temp_df['weekday']==2)&(temp_df['hour']==16))).astype(int)
    tests[num-1] = temp_df
for num in [1]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=7)).astype(int)
    temp_df['low_day'] = (temp_df['weekday']==6).astype(int)
    tests[num-1] = temp_df
for num in [8]:
    temp_df = tests[num-1]
    temp_df['low_day'] = (temp_df['weekday']==6).astype(int)
    tests[num-1] = temp_df
for num in [12]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=5)).astype(int)
    temp_df['low_day'] = (temp_df['weekday']==6).astype(int)
    tests[num-1] = temp_df
for num in [88,90]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=22)&(temp_df['hour']>=8)).astype(int)
    tests[num-1] = temp_df
for num in [86]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=22)&(temp_df['hour']>=8)).astype(int)
    temp_df['particular'] = ((temp_df['month']==6)&(temp_df['day']==26)).astype(int)
    tests[num-1] = temp_df
for num in [87,89,91]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=9)).astype(int)
    tests[num-1] = temp_df
for num in [4]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=18)&(temp_df['hour']>=8)).astype(int)
    tests[num-1] = temp_df
for num in [7,16]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=22)&(temp_df['hour']>=5)).astype(int)
    tests[num-1] = temp_df
for num in [9]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=23)&(temp_df['hour']>=7)).astype(int)
    tests[num-1] = temp_df
for num in [11]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=21)&(temp_df['hour']>=7)).astype(int)
    tests[num-1] = temp_df
for num in [36]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=20)&(temp_df['hour']>=9)).astype(int)
    tests[num-1] = temp_df
for num in [75]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=15)&(temp_df['hour']>=7)).astype(int)
    tests[num-1] = temp_df
for num in [92]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']<=22)&(temp_df['hour']>=6)).astype(int)
    tests[num-1] = temp_df

## cluster 2
for num in [32,33,34]:
    temp_df = tests[num-1]
    temp_df = temp_df[temp_df.month>=7]
    tests[num-1] = temp_df
for num in [61,65,66,67,68]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=7)&(temp_df['hour']<=23)).astype(int)
    tests[num-1] = temp_df
for num in [62,63,64]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=8)&(temp_df['hour']<=23)).astype(int)
    tests[num-1] = temp_df

## cluster 3
for num in [93,94,96,97,98,100]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=6)&(temp_df['hour']<=22)).astype(int)
    tests[num-1] = temp_df
for num in [42,43,44]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=9)&(temp_df['hour']<=20)).astype(int)
    tests[num-1] = temp_df
for num in [2,85,99]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=8)&(temp_df['hour']<=21)).astype(int)
    tests[num-1] = temp_df
for num in [38,40,95]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=8)&(temp_df['hour']<=20)).astype(int)
    tests[num-1] = temp_df
for num in [3,81]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=9)&(temp_df['hour']<=21)).astype(int)
    tests[num-1] = temp_df
for num in [6,41]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=8)&(temp_df['hour']<=22)).astype(int)
    tests[num-1] = temp_df
for num in [37,39]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=7)&(temp_df['hour']<=21)).astype(int)
    tests[num-1] = temp_df
for num in [35]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=7)&(temp_df['hour']<=23)).astype(int)
    tests[num-1] = temp_df
for num in [54]:
    temp_df = tests[num-1]
    temp_df['work_time'] = ((temp_df['hour']>=5)&(temp_df['hour']<=20)).astype(int)
    temp_df['low_day'] = (temp_df['weekday']==0).astype(int)
    tests[num-1] = temp_df
for num in [5]:
    temp_df = tests[num-1]
    temp_df['particular'] = (((temp_df['weekday']==4)&(temp_df['hour'].isin([11,12,13,14,15,16,17,18,19,20])))|
                              ((temp_df['weekday']==5)&(temp_df['hour'].isin([11,12,13,14,15,16,17,18,19,20])))|
                              ((temp_df['weekday']==6)&(temp_df['hour'].isin([8,9,10,11,12,13,14,15,16,17,18])))).astype(int)
    tests[num-1] = temp_df
    

df = pd.concat(tests, ignore_index=True)
df = df.drop(['date_time','hour','day','month','weekday','date'], axis=1)
df.building_type = df.building_type.astype('category')

df_0 = df[df.km_cluster==0]
df_1 = df[df.km_cluster==1]
df_2 = df[df.km_cluster==2]
df_3 = df[df.km_cluster==3]

df_0 = pd.get_dummies(df_0)
df_1 = pd.get_dummies(df_1)
df_2 = pd.get_dummies(df_2)
df_3 = pd.get_dummies(df_3)

X_0 = df_0.drop('km_cluster', axis=1)
X_1 = df_1.drop('km_cluster', axis=1)
X_2 = df_2.drop('km_cluster', axis=1)
X_3 = df_3.drop('km_cluster', axis=1)


# Standard Scaling for continuous data
con_li = ['hour_sin', 'hour_cos', 'date_sin', 'date_cos', 'month_sin', 'month_cos', 'weekday_sin', 'weekday_cos', 'temperature', 'windspeed', 
'humidity', 'total_area', 'cooling_area', 'solar_power_capacity', 'ess_capacity', 'pcs_capacity', 'discomfort', 'temperature_3', 
'discomfort_3', 'ma_dis_3', 'ma_dis_5', 'CDH', 'temperature_mean', 'discomfort_mean', 'CDH_mean', 'sensory_temperature']


cat_li = ['building_number','holiday','air_yes','solar_yes',
         'ess_yes','sy_ey','sy_en','low_day','particular']


for col in con_li :
    if X_0[col].std() == 0:
        X_0 = X_0.drop(col, axis=1)
    else :
        X_0[col] = (X_0[col] - X_0[col].mean())/X_0[col].std()
        
for col in cat_li :
    if X_0[col].nunique() == 1:        
        X_0 = X_0.drop(col,axis=1)
X_0_test = X_0

for col in con_li :
    if X_1[col].std() == 0:
        X_1 = X_1.drop(col, axis=1)
    else :
        X_1[col] = (X_1[col] - X_1[col].mean())/X_1[col].std()
        
for col in cat_li :
    if X_1[col].nunique() == 1:
        X_1 = X_1.drop(col,axis=1)
X_1_test = X_1

for col in con_li :
    if X_2[col].std() == 0:
        X_2 = X_2.drop(col, axis=1)
    else :
        X_2[col] = (X_2[col] - X_2[col].mean())/X_2[col].std()
        
for col in cat_li :
    if X_2[col].nunique() == 1:
        X_2 = X_2.drop(col,axis=1)
X_2_test = X_2
for col in con_li :
    if X_3[col].std() == 0:
        X_3 = X_3.drop(col, axis=1)
    else :
        X_3[col] = (X_3[col] - X_3[col].mean())/X_3[col].std()
        
for col in cat_li :
    if X_3[col].nunique() == 1:
        X_3 = X_3.drop(col,axis=1)
X_3_test = X_3

col_0 = ['building_number','date_sin','hour_cos','work_time','hour_sin','holiday','weekday_cos','discomfort_mean','particular','CDH',
'temperature_mean','discomfort','weekday_sin','CDH_mean','solar_power_capacity']
col_1 = ['cooling_area','building_number','work_time','total_area','hour_cos','hour_sin','holiday','date_sin','building_type_Hospital',
        'low_day','solar_power_capacity','building_type_University','building_type_Research Institute','building_type_Commercial',
        'weekday_cos','sy_ey']
col_2 = ['building_number','total_area','work_time','hour_sin','cooling_area','date_sin','ma_dis_5','CDH','hour_cos']
col_3 = ['work_time','total_area','building_number','hour_cos','hour_sin','building_type_Department Store and Outlet','date_sin',
        'cooling_area','discomfort','particular','solar_power_capacity']

X_0_train = X_0_train[col_0]
X_1_train = X_1_train[col_1]
X_2_train = X_2_train[col_2]
X_3_train = X_3_train[col_3]



C:\Users\yeji4\AppData\Local\Temp/ipykernel_12252/3850810787.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['work_time'] = (temp_df['hour'].isin([6,7,8])).astype(int)
C:\Users\yeji4\AppData\Local\Temp/ipykernel_12252/3850810787.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['work_time'] = (temp_df['hour'].isin([6,7])).astype(int)
C:\Users\yeji4\AppData\Local\Temp/ipykernel_12252/3850810787.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [ ]:
#### xgb for ensemble ####

xgb_best_scores = []
xgb_hyperparams = []

## cluster 0 model
X_train, X_valid, y_train, y_valid = train_test_split(X_0_train, y_0_log, random_state=42)

def objective(trial):
    param = {
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1),
        'subsample': trial.suggest_float('subsample', 0.4, 1),
        'learning_rate': trial.suggest_float('learning_rate',0.001, 0.01),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 4,8),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 50),
    }
    model =XGBRegressor(**param)  
    
    model.fit(X_train,y_train,eval_set=[(X_valid,y_valid)],early_stopping_rounds=50)
    
    preds = model.predict(X_valid)
    
    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm
          
study_0 = optuna.create_study(study_name = 'Xgboost Optuna_0')
study_0.optimize(objective, n_trials=50, timeout=24000)

xgb_0 = study_0.best_trial
xgb_0_param = xgb_0.params
print('Best Trial: score {},\nparams {}'.format(xgb_0.value, xgb_0_param))

xgb_best_scores.append(xgb_0)
xgb_hyperparams.append(xgb_0_param)

## cluster 1 model
X_train, X_valid, y_train, y_valid = train_test_split(X_1_train, y_1_log, random_state=42)

def objective(trial):
    param = {
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1),
        'subsample': trial.suggest_float('subsample', 0.4, 1),
        'learning_rate': trial.suggest_float('learning_rate',0.001, 0.01),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 4,8),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 50),
    }
    model =XGBRegressor(**param)  
    
    model.fit(X_train,y_train,eval_set=[(X_valid,y_valid)],early_stopping_rounds=50)
    
    preds = model.predict(X_valid)
    
    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm
          
study_1 = optuna.create_study(study_name = 'Xgboost Optuna_1')
study_1.optimize(objective, n_trials=50, timeout=24000)

xgb_1 = study_1.best_trial
xgb_1_param = xgb_1.params
print('Best Trial: score {},\nparams {}'.format(xgb_1.value, xgb_1_param))

xgb_best_scores.append(xgb_1)
xgb_hyperparams.append(xgb_1_param)

## cluster 2 model
X_train, X_valid, y_train, y_valid = train_test_split(X_2_train, y_2_log, random_state=42)

def objective(trial):
    param = {
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1),
        'subsample': trial.suggest_float('subsample', 0.4, 1),
        'learning_rate': trial.suggest_float('learning_rate',0.001, 0.01),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 4,8),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 50),
    }
    model =XGBRegressor(**param)  
    
    model.fit(X_train,y_train,eval_set=[(X_valid,y_valid)],early_stopping_rounds=50)
    
    preds = model.predict(X_valid)
    
    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm
          
study_2 = optuna.create_study(study_name = 'Xgboost Optuna_2')
study_2.optimize(objective, n_trials=50, timeout=24000)

xgb_2 = study_2.best_trial
xgb_2_param = xgb_2.params
print('Best Trial: score {},\nparams {}'.format(xgb_2.value, xgb_2_param))

xgb_best_scores.append(xgb_2)
xgb_hyperparams.append(xgb_2_param)

## cluster 3 model
X_train, X_valid, y_train, y_valid = train_test_split(X_3_train, y_3_log, random_state=42)

def objective(trial):
    param = {
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1),
        'subsample': trial.suggest_float('subsample', 0.4, 1),
        'learning_rate': trial.suggest_float('learning_rate',0.001, 0.01),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 4,8),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 50),
    }
    model =XGBRegressor(**param)  
    
    model.fit(X_train,y_train,eval_set=[(X_valid,y_valid)],early_stopping_rounds=50)
    
    preds = model.predict(X_valid)
    
    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm
          
study_3 = optuna.create_study(study_name = 'Xgboost Optuna_3')
study_3.optimize(objective, n_trials=50, timeout=24000)

xgb_3 = study_3.best_trial
xgb_3_param = xgb_3.params
print('Best Trial: score {},\nparams {}'.format(xgb_3.value, xgb_3_param))

xgb_best_scores.append(xgb_3)
xgb_hyperparams.append(xgb_3_param)

In [3]:
xgb_best_scores

[FrozenTrial(number=10, state=TrialState.COMPLETE, values=[9.786584354730003], datetime_start=datetime.datetime(2023, 8, 18, 15, 23, 14, 289445), datetime_complete=datetime.datetime(2023, 8, 18, 15, 23, 15, 561304), params={'colsample_bytree': 0.5612041017618803, 'subsample': 0.6139630768428379, 'learning_rate': 0.009622630932242778, 'n_estimators': 497, 'max_depth': 8, 'min_child_weight': 21}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'subsample': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'learning_rate': FloatDistribution(high=0.01, log=False, low=0.001, step=None), 'n_estimators': IntDistribution(high=500, log=False, low=100, step=1), 'max_depth': IntDistribution(high=8, log=False, low=4, step=1), 'min_child_weight': IntDistribution(high=50, log=False, low=2, step=1)}, trial_id=10, value=None),
 FrozenTrial(number=27, state=TrialState.COMPLETE, values=[12.03

In [4]:
xgb_hyperparams

[{'colsample_bytree': 0.5612041017618803,
  'subsample': 0.6139630768428379,
  'learning_rate': 0.009622630932242778,
  'n_estimators': 497,
  'max_depth': 8,
  'min_child_weight': 21},
 {'colsample_bytree': 0.8773856909719138,
  'subsample': 0.9435438002329848,
  'learning_rate': 0.00925564912369709,
  'n_estimators': 461,
  'max_depth': 8,
  'min_child_weight': 20},
 {'colsample_bytree': 0.9697076211677187,
  'subsample': 0.9326141056879428,
  'learning_rate': 0.009995749790295971,
  'n_estimators': 500,
  'max_depth': 7,
  'min_child_weight': 4},
 {'colsample_bytree': 0.7223565317608216,
  'subsample': 0.7349834768081919,
  'learning_rate': 0.009921834484899176,
  'n_estimators': 462,
  'max_depth': 6,
  'min_child_weight': 46}]

In [5]:
xgb_best_scores[0]

FrozenTrial(number=10, state=TrialState.COMPLETE, values=[9.786584354730003], datetime_start=datetime.datetime(2023, 8, 18, 15, 23, 14, 289445), datetime_complete=datetime.datetime(2023, 8, 18, 15, 23, 15, 561304), params={'colsample_bytree': 0.5612041017618803, 'subsample': 0.6139630768428379, 'learning_rate': 0.009622630932242778, 'n_estimators': 497, 'max_depth': 8, 'min_child_weight': 21}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'subsample': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'learning_rate': FloatDistribution(high=0.01, log=False, low=0.001, step=None), 'n_estimators': IntDistribution(high=500, log=False, low=100, step=1), 'max_depth': IntDistribution(high=8, log=False, low=4, step=1), 'min_child_weight': IntDistribution(high=50, log=False, low=2, step=1)}, trial_id=10, value=None)

In [6]:
xgb_0.value

9.786584354730003

In [9]:
xgb_3.value

12.590079430048801

In [11]:
pip install catboost

     -------------------------------------- 101.0/101.0 MB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 47.0/47.0 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


# Catboost

## Catboost : cluster 0

In [4]:
#### cat for ensemble ####

from sklearn.model_selection import train_test_split
import optuna
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
import math


cat_best_scores = []
cat_hyperparams = []


In [46]:

## cluster 0 model
X_0_train = X_0_train[col_0]
X_0_test = X_0_test[col_0]

X_train, X_valid, y_train, y_valid = train_test_split(X_0_train, y_0_log)

def objective(trial):
    param = {}
    param['learning_rate'] = trial.suggest_float("learning_rate", 1e-3, 0.1, log=True)
    param['depth'] = trial.suggest_int('depth', 10, 16)
    param['l2_leaf_reg'] = trial.suggest_float('l2_leaf_reg',low=1.0, high=5.5, step=0.5)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16])
    param['grow_policy'] = 'Depthwise'
    param['iterations'] = 1000
    param['eval_metric'] = 'SMAPE'
    param['od_type'] = 'iter'
    param['od_wait'] = 20
    param['random_state'] = 1
    param['logging_level'] = 'Silent'
    regressor = CatBoostRegressor(**param)
    regressor.fit(X_train, y_train,early_stopping_rounds=150)
    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(regressor.predict(X_valid))))
    return sm

study_0 = optuna.create_study(study_name=f'catboost-seed')
study_0.optimize(objective, n_trials=50, n_jobs=-1, timeout=24000)

cat_0 = study_0.best_trial
cat_0_param = cat_0.params
print('Best Trial: score {},\nparams {}'.format(cat_0.value, cat_0_param))

cat_best_scores.append(cat_0.value)
cat_hyperparams.append(cat_0_param)



[I 2023-08-19 09:59:40,868] A new study created in memory with name: catboost-seed
[I 2023-08-19 10:00:33,316] Trial 5 finished with value: 4.478263492335539 and parameters: {'learning_rate': 0.021507602952939915, 'depth': 11, 'l2_leaf_reg': 1.5, 'min_child_samples': 8}. Best is trial 5 with value: 4.478263492335539.
[I 2023-08-19 10:00:49,903] Trial 1 finished with value: 4.380765386100521 and parameters: {'learning_rate': 0.03093153036022651, 'depth': 15, 'l2_leaf_reg': 2.5, 'min_child_samples': 16}. Best is trial 1 with value: 4.380765386100521.
[I 2023-08-19 10:00:51,006] Trial 4 finished with value: 4.628883233980192 and parameters: {'learning_rate': 0.02826378216261558, 'depth': 11, 'l2_leaf_reg': 5.0, 'min_child_samples': 4}. Best is trial 1 with value: 4.380765386100521.
[I 2023-08-19 10:02:29,075] Trial 3 finished with value: 4.591597027654483 and parameters: {'learning_rate': 0.015427440510497844, 'depth': 12, 'l2_leaf_reg': 3.0, 'min_child_samples': 4}. Best is trial 1 with 

[I 2023-08-19 10:22:22,752] Trial 34 finished with value: 4.4214044292413135 and parameters: {'learning_rate': 0.054992984607162065, 'depth': 14, 'l2_leaf_reg': 1.0, 'min_child_samples': 1}. Best is trial 29 with value: 4.3413967925355745.
[I 2023-08-19 10:22:51,686] Trial 23 finished with value: 4.567373246887975 and parameters: {'learning_rate': 0.0670330975860676, 'depth': 16, 'l2_leaf_reg': 1.5, 'min_child_samples': 1}. Best is trial 29 with value: 4.3413967925355745.
[I 2023-08-19 10:23:10,303] Trial 40 finished with value: 4.378284064801224 and parameters: {'learning_rate': 0.03898978349749659, 'depth': 14, 'l2_leaf_reg': 1.0, 'min_child_samples': 16}. Best is trial 29 with value: 4.3413967925355745.
[I 2023-08-19 10:23:13,891] Trial 35 finished with value: 4.412335831856703 and parameters: {'learning_rate': 0.04374033773568721, 'depth': 14, 'l2_leaf_reg': 1.0, 'min_child_samples': 1}. Best is trial 29 with value: 4.3413967925355745.
[I 2023-08-19 10:23:17,646] Trial 41 finished 

Best Trial: score 4.3413967925355745,
params {'learning_rate': 0.05166676124789215, 'depth': 15, 'l2_leaf_reg': 2.5, 'min_child_samples': 16}


## Catboost : cluster 1

In [10]:
## cluster 1 model
X_1_train = X_1_train[col_1]
X_1_test = X_1_test[col_1]

X_train, X_valid, y_train, y_valid = train_test_split(X_1_train, y_1_log)

def objective(trial):
    param = {}
    param['learning_rate'] = trial.suggest_float("learning_rate", 0.01, 0.1, log=True)
    param['depth'] = trial.suggest_int('depth', 10, 16)
    param['l2_leaf_reg'] = trial.suggest_float('l2_leaf_reg',low=2.0, high=5.5, step=0.5)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8])
    param['grow_policy'] = 'Depthwise'
    param['iterations'] = 1000
    param['eval_metric'] = 'SMAPE'
    param['od_type'] = 'iter'
    param['od_wait'] = 20
    param['random_state'] = 1
    param['logging_level'] = 'Silent'
    regressor = CatBoostRegressor(**param)
    regressor.fit(X_train, y_train,early_stopping_rounds=150)
    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(regressor.predict(X_valid))))
    return sm

study_1 = optuna.create_study(study_name=f'catboost-seed')
study_1.optimize(objective, n_trials=50, n_jobs=-1, timeout=24000)

cat_1 = study_1.best_trial
cat_1_param = cat_1.params
print('Best Trial: score {},\nparams {}'.format(cat_1.value, cat_1_param))

cat_best_scores.append(cat_1.value)
cat_hyperparams.append(cat_1_param)



[I 2023-08-19 14:11:42,371] A new study created in memory with name: catboost-seed
[I 2023-08-19 14:23:47,252] Trial 1 finished with value: 2.9732379602330323 and parameters: {'learning_rate': 0.051053606627558215, 'depth': 10, 'l2_leaf_reg': 2.0, 'min_child_samples': 8}. Best is trial 1 with value: 2.9732379602330323.
[I 2023-08-19 14:30:06,256] Trial 0 finished with value: 3.3191145005200173 and parameters: {'learning_rate': 0.023685387299609985, 'depth': 11, 'l2_leaf_reg': 4.5, 'min_child_samples': 8}. Best is trial 1 with value: 2.9732379602330323.
[I 2023-08-19 14:47:07,780] Trial 5 finished with value: 2.6301328162092736 and parameters: {'learning_rate': 0.06982381742778836, 'depth': 12, 'l2_leaf_reg': 2.5, 'min_child_samples': 4}. Best is trial 5 with value: 2.6301328162092736.
[I 2023-08-19 14:48:47,486] Trial 4 finished with value: 2.93003403688157 and parameters: {'learning_rate': 0.025341698714175076, 'depth': 11, 'l2_leaf_reg': 2.0, 'min_child_samples': 1}. Best is trial 5 

Best Trial: score 2.426650381695719,
params {'learning_rate': 0.047386859888027516, 'depth': 16, 'l2_leaf_reg': 3.5, 'min_child_samples': 1}


## Catboost : cluster 2

In [11]:
## cluster 2 model
X_2_train = X_2_train[col_2]
X_2_test = X_2_test[col_2]

X_train, X_valid, y_train, y_valid = train_test_split(X_2_train, y_2_log)

def objective(trial):
    param = {}
    param['learning_rate'] = trial.suggest_float("learning_rate", 0.01, 0.1, log=True)
    param['depth'] = trial.suggest_int('depth', 10, 16)
    param['l2_leaf_reg'] = trial.suggest_float('l2_leaf_reg',low=2.0, high=5.5, step=0.5)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8])
    param['grow_policy'] = 'Depthwise'
    param['iterations'] = 1000
    param['eval_metric'] = 'SMAPE'
    param['od_type'] = 'iter'
    param['od_wait'] = 20
    param['random_state'] = 1
    param['logging_level'] = 'Silent'
    regressor = CatBoostRegressor(**param)
    regressor.fit(X_train, y_train,early_stopping_rounds=150)
    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(regressor.predict(X_valid))))
    return sm

study_2 = optuna.create_study(study_name=f'catboost-seed')
study_2.optimize(objective, n_trials=50, n_jobs=-1, timeout=12000)

cat_2 = study_2.best_trial
cat_2_param = cat_2.params
print('Best Trial: score {},\nparams {}'.format(cat_2.value, cat_2_param))

cat_best_scores.append(cat_2.value)
cat_hyperparams.append(cat_2_param)



[I 2023-08-20 12:22:26,169] A new study created in memory with name: catboost-seed
[I 2023-08-20 12:24:29,134] Trial 6 finished with value: 2.7510827661555255 and parameters: {'learning_rate': 0.011498394043737573, 'depth': 10, 'l2_leaf_reg': 2.5, 'min_child_samples': 1}. Best is trial 6 with value: 2.7510827661555255.
[I 2023-08-20 12:25:06,189] Trial 4 finished with value: 2.2852857810166016 and parameters: {'learning_rate': 0.0415824551730397, 'depth': 13, 'l2_leaf_reg': 3.5, 'min_child_samples': 8}. Best is trial 4 with value: 2.2852857810166016.
[I 2023-08-20 12:25:07,928] Trial 1 finished with value: 2.4359994695941056 and parameters: {'learning_rate': 0.08223069661582878, 'depth': 14, 'l2_leaf_reg': 2.0, 'min_child_samples': 8}. Best is trial 4 with value: 2.2852857810166016.
[I 2023-08-20 12:26:51,853] Trial 7 finished with value: 2.345451229972235 and parameters: {'learning_rate': 0.04623582830693794, 'depth': 15, 'l2_leaf_reg': 2.5, 'min_child_samples': 8}. Best is trial 4 wi

[I 2023-08-20 12:54:05,434] Trial 40 finished with value: 2.3215341192100296 and parameters: {'learning_rate': 0.037344116753045616, 'depth': 10, 'l2_leaf_reg': 5.0, 'min_child_samples': 1}. Best is trial 12 with value: 2.229662738410891.
[I 2023-08-20 12:55:08,067] Trial 41 finished with value: 2.2283386341737703 and parameters: {'learning_rate': 0.035694134425092706, 'depth': 10, 'l2_leaf_reg': 2.0, 'min_child_samples': 1}. Best is trial 41 with value: 2.2283386341737703.
[I 2023-08-20 12:55:29,677] Trial 43 finished with value: 2.227388494840722 and parameters: {'learning_rate': 0.04391319676126841, 'depth': 11, 'l2_leaf_reg': 2.0, 'min_child_samples': 8}. Best is trial 43 with value: 2.227388494840722.
[I 2023-08-20 12:55:31,913] Trial 42 finished with value: 2.2203778156553087 and parameters: {'learning_rate': 0.04406610508389325, 'depth': 10, 'l2_leaf_reg': 2.0, 'min_child_samples': 1}. Best is trial 42 with value: 2.2203778156553087.
[I 2023-08-20 12:55:33,359] Trial 25 finished

Best Trial: score 2.219162575294043,
params {'learning_rate': 0.04314674364434326, 'depth': 11, 'l2_leaf_reg': 2.0, 'min_child_samples': 8}


## Catboost : cluster 3

In [5]:
## cluster 3 model
X_3_train = X_3_train[col_3]
X_3_test = X_3_test[col_3]

X_train, X_valid, y_train, y_valid = train_test_split(X_3_train, y_3_log)

def objective(trial):
    param = {}
    param['learning_rate'] = trial.suggest_float("learning_rate", 0.01, 0.1, log=True)
    param['depth'] = trial.suggest_int('depth', 10, 16)
    param['l2_leaf_reg'] = trial.suggest_float('l2_leaf_reg',low=2.0, high=5.5, step=0.5)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8])
    param['grow_policy'] = 'Depthwise'
    param['iterations'] = 1000
    param['eval_metric'] = 'SMAPE'
    param['od_type'] = 'iter'
    param['od_wait'] = 20
    param['random_state'] = 1
    param['logging_level'] = 'Silent'
    regressor = CatBoostRegressor(**param)
    regressor.fit(X_train, y_train,early_stopping_rounds=150)
    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(regressor.predict(X_valid))))
    return sm

study_3 = optuna.create_study(study_name=f'catboost-seed')
study_3.optimize(objective, n_trials=50, n_jobs=-1, timeout=12000)

cat_3 = study_3.best_trial
cat_3_param = cat_3.params
print('Best Trial: score {},\nparams {}'.format(cat_3.value, cat_3_param))

cat_best_scores.append(cat_3.value)
cat_hyperparams.append(cat_3_param)

[I 2023-08-20 13:05:04,583] A new study created in memory with name: catboost-seed
[I 2023-08-20 13:06:06,123] Trial 7 finished with value: 5.345104463660827 and parameters: {'learning_rate': 0.05233549566695084, 'depth': 10, 'l2_leaf_reg': 5.0, 'min_child_samples': 4}. Best is trial 7 with value: 5.345104463660827.
[I 2023-08-20 13:06:41,886] Trial 4 finished with value: 5.052491398428096 and parameters: {'learning_rate': 0.08855455552533321, 'depth': 11, 'l2_leaf_reg': 2.0, 'min_child_samples': 4}. Best is trial 4 with value: 5.052491398428096.
[I 2023-08-20 13:06:46,731] Trial 1 finished with value: 5.024908969009803 and parameters: {'learning_rate': 0.06613475931331063, 'depth': 11, 'l2_leaf_reg': 2.0, 'min_child_samples': 4}. Best is trial 1 with value: 5.024908969009803.
[I 2023-08-20 13:07:59,731] Trial 6 finished with value: 5.039587563139891 and parameters: {'learning_rate': 0.0477404329091495, 'depth': 13, 'l2_leaf_reg': 5.5, 'min_child_samples': 8}. Best is trial 1 with valu

[I 2023-08-20 16:42:04,818] Trial 35 finished with value: 4.772628021588889 and parameters: {'learning_rate': 0.054127338414346304, 'depth': 15, 'l2_leaf_reg': 3.5, 'min_child_samples': 1}. Best is trial 30 with value: 4.766026429169772.
[I 2023-08-20 16:43:28,559] Trial 36 finished with value: 4.7971104660416115 and parameters: {'learning_rate': 0.05037082060656059, 'depth': 15, 'l2_leaf_reg': 4.5, 'min_child_samples': 1}. Best is trial 30 with value: 4.766026429169772.
[I 2023-08-20 16:47:55,025] Trial 37 finished with value: 4.775982370713721 and parameters: {'learning_rate': 0.052277667915279546, 'depth': 15, 'l2_leaf_reg': 4.5, 'min_child_samples': 1}. Best is trial 30 with value: 4.766026429169772.
[I 2023-08-20 16:50:44,802] Trial 38 finished with value: 4.8156214029395645 and parameters: {'learning_rate': 0.052676869856488416, 'depth': 15, 'l2_leaf_reg': 4.5, 'min_child_samples': 1}. Best is trial 30 with value: 4.766026429169772.
[I 2023-08-20 16:52:28,820] Trial 39 finished w

Best Trial: score 4.766026429169772,
params {'learning_rate': 0.06230456276497763, 'depth': 15, 'l2_leaf_reg': 3.5, 'min_child_samples': 1}


In [17]:
cat_hyperparams

[{'learning_rate': 0.07150105814625922,
  'depth': 10,
  'l2_leaf_reg': 2.5,
  'min_child_samples': 4},
 {'learning_rate': 0.09503521045667257,
  'depth': 10,
  'l2_leaf_reg': 2.0,
  'min_child_samples': 4},
 {'learning_rate': 0.05605488347126201,
  'depth': 10,
  'l2_leaf_reg': 3.0,
  'min_child_samples': 1},
 {'learning_rate': 0.09889161706735772,
  'depth': 10,
  'l2_leaf_reg': 4.5,
  'min_child_samples': 1}]

In [13]:
cat_1.value

2.7849333604938664

In [14]:
cat_2.value

2.1866003917783434

In [15]:
cat_3.value

5.018264109863932

In [7]:
cat_best_scores

[4.766026429169772]

# RF

In [25]:
#### rf for ensemble ####
from sklearn.ensemble import RandomForestRegressor

rf_best_scores = []
rf_hyperparams = []

## cluster 0
X_train, X_valid, y_train, y_valid = train_test_split(X_0_train, y_0_log, random_state=42)

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'criterion': 'squared_error',
        'max_depth' : trial.suggest_int('max_depth', 4, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 3, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3, 10),
        'random_state': 28
    }

    model = RandomForestRegressor(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm

study_0 = optuna.create_study(study_name = 'RF Optuna_0')
study_0.optimize(objective, n_trials=50, timeout=24000)

rf_0 = study_0.best_trial
rf_0_param = rf_0.params
print('Best Trial: score {},\nparams {}'.format(rf_0.value, rf_0_param))

rf_best_scores.append(rf_0.value)
rf_hyperparams.append(rf_0_param)

## cluster 1
X_train, X_valid, y_train, y_valid = train_test_split(X_1_train, y_1_log, random_state=42)

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'criterion': 'squared_error',
        'max_depth' : trial.suggest_int('max_depth', 4, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 3, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3, 10),
        'random_state': 28
    }

    model = RandomForestRegressor(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm

study_1 = optuna.create_study(study_name = 'RF Optuna_1')
study_1.optimize(objective, n_trials=50, timeout=24000)

rf_1 = study_1.best_trial
rf_1_param = rf_1.params
print('Best Trial: score {},\nparams {}'.format(rf_1.value, rf_1_param))

rf_best_scores.append(rf_1.value)
rf_hyperparams.append(rf_1_param)

## cluster 2
X_train, X_valid, y_train, y_valid = train_test_split(X_2_train, y_2_log, random_state=42)

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'criterion': 'squared_error',
        'max_depth' : trial.suggest_int('max_depth', 4, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 3, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3, 10),
        'random_state': 28
    }

    model = RandomForestRegressor(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm

study_2 = optuna.create_study(study_name = 'RF Optuna_2')
study_2.optimize(objective, n_trials=50, timeout=24000)

rf_2 = study_2.best_trial
rf_2_param = rf_2.params
print('Best Trial: score {},\nparams {}'.format(rf_2.value, rf_2_param))

rf_best_scores.append(rf_2.value)
rf_hyperparams.append(rf_2_param)

## cluster 3
X_train, X_valid, y_train, y_valid = train_test_split(X_3_train, y_3_log, random_state=42)

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'criterion': 'squared_error',
        'max_depth' : trial.suggest_int('max_depth', 4, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 3, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3, 10),
        'random_state': 28
    }

    model = RandomForestRegressor(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm

study_3 = optuna.create_study(study_name = 'RF Optuna_3')
study_3.optimize(objective, n_trials=50, timeout=24000)

rf_3 = study_3.best_trial
rf_3_param = rf_3.params
print('Best Trial: score {},\nparams {}'.format(rf_3.value, rf_3_param))

rf_best_scores.append(rf_3.value)
rf_hyperparams.append(rf_3_param)


[I 2023-08-18 19:50:17,832] A new study created in memory with name: RF Optuna_0
[I 2023-08-18 19:50:25,689] Trial 0 finished with value: 6.429082832136099 and parameters: {'n_estimators': 460, 'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 0 with value: 6.429082832136099.
[I 2023-08-18 19:50:29,178] Trial 1 finished with value: 6.656769471490162 and parameters: {'n_estimators': 226, 'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 7}. Best is trial 0 with value: 6.429082832136099.
[I 2023-08-18 19:50:34,895] Trial 2 finished with value: 6.753280328590118 and parameters: {'n_estimators': 381, 'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 9}. Best is trial 0 with value: 6.429082832136099.
[I 2023-08-18 19:50:41,751] Trial 3 finished with value: 7.345739499789489 and parameters: {'n_estimators': 500, 'max_depth': 8, 'min_samples_split': 5, 'min_samples_leaf': 4}. Best is trial 0 with value: 6.429082832136099.
[I 2023-08-18 19:50:

[I 2023-08-18 19:53:01,749] Trial 36 finished with value: 6.482500753747969 and parameters: {'n_estimators': 480, 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 3}. Best is trial 0 with value: 6.429082832136099.
[I 2023-08-18 19:53:08,995] Trial 37 finished with value: 6.882534774265525 and parameters: {'n_estimators': 478, 'max_depth': 9, 'min_samples_split': 10, 'min_samples_leaf': 3}. Best is trial 0 with value: 6.429082832136099.
[I 2023-08-18 19:53:14,610] Trial 38 finished with value: 7.350668266109894 and parameters: {'n_estimators': 410, 'max_depth': 8, 'min_samples_split': 10, 'min_samples_leaf': 3}. Best is trial 0 with value: 6.429082832136099.
[I 2023-08-18 19:53:18,030] Trial 39 finished with value: 10.284697591929648 and parameters: {'n_estimators': 464, 'max_depth': 4, 'min_samples_split': 7, 'min_samples_leaf': 3}. Best is trial 0 with value: 6.429082832136099.
[I 2023-08-18 19:53:25,413] Trial 40 finished with value: 6.466485235069077 and parameters: {'n

Best Trial: score 6.429082832136099,
params {'n_estimators': 460, 'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 3}


[I 2023-08-18 19:55:00,354] Trial 0 finished with value: 13.780258852338726 and parameters: {'n_estimators': 404, 'max_depth': 8, 'min_samples_split': 10, 'min_samples_leaf': 6}. Best is trial 0 with value: 13.780258852338726.
[I 2023-08-18 19:55:16,476] Trial 1 finished with value: 11.437061383878962 and parameters: {'n_estimators': 166, 'max_depth': 9, 'min_samples_split': 3, 'min_samples_leaf': 3}. Best is trial 1 with value: 11.437061383878962.
[I 2023-08-18 19:55:54,561] Trial 2 finished with value: 11.462348787216953 and parameters: {'n_estimators': 392, 'max_depth': 9, 'min_samples_split': 8, 'min_samples_leaf': 6}. Best is trial 1 with value: 11.437061383878962.
[I 2023-08-18 19:56:40,304] Trial 3 finished with value: 11.470155327925525 and parameters: {'n_estimators': 468, 'max_depth': 9, 'min_samples_split': 5, 'min_samples_leaf': 7}. Best is trial 1 with value: 11.437061383878962.
[I 2023-08-18 19:57:04,463] Trial 4 finished with value: 30.777415953054778 and parameters: {'n

[I 2023-08-18 20:08:16,477] Trial 37 finished with value: 11.43780745210523 and parameters: {'n_estimators': 167, 'max_depth': 9, 'min_samples_split': 7, 'min_samples_leaf': 3}. Best is trial 9 with value: 9.791609186544664.
[I 2023-08-18 20:08:44,787] Trial 38 finished with value: 9.813744443881243 and parameters: {'n_estimators': 271, 'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 4}. Best is trial 9 with value: 9.791609186544664.
[I 2023-08-18 20:08:52,300] Trial 39 finished with value: 30.78158179267397 and parameters: {'n_estimators': 148, 'max_depth': 4, 'min_samples_split': 3, 'min_samples_leaf': 3}. Best is trial 9 with value: 9.791609186544664.
[I 2023-08-18 20:09:03,354] Trial 40 finished with value: 13.768198310285074 and parameters: {'n_estimators': 125, 'max_depth': 8, 'min_samples_split': 10, 'min_samples_leaf': 5}. Best is trial 9 with value: 9.791609186544664.
[I 2023-08-18 20:09:24,302] Trial 41 finished with value: 9.817650616388034 and parameters: {'n_e

...

[I 2023-08-18 20:16:25,173] Trial 37 finished with value: 6.016994882618078 and parameters: {'n_estimators': 196, 'max_depth': 9, 'min_samples_split': 3, 'min_samples_leaf': 4}. Best is trial 36 with value: 5.320112655350761.
[I 2023-08-18 20:16:30,015] Trial 38 finished with value: 5.304094123378454 and parameters: {'n_estimators': 157, 'max_depth': 10, 'min_samples_split': 3, 'min_samples_leaf': 3}. Best is trial 38 with value: 5.304094123378454.
[I 2023-08-18 20:16:38,329] Trial 39 finished with value: 7.044670153740157 and parameters: {'n_estimators': 331, 'max_depth': 8, 'min_samples_split': 3, 'min_samples_leaf': 3}. Best is trial 38 with value: 5.304094123378454.
[I 2023-08-18 20:16:41,342] Trial 40 finished with value: 10.068861421814383 and parameters: {'n_estimators': 159, 'max_depth': 6, 'min_samples_split': 3, 'min_samples_leaf': 3}. Best is trial 38 with value: 5.304094123378454.
[I 2023-08-18 20:16:45,189] Trial 41 finished with value: 5.314330826456314 and parameters: {'

Best Trial: score 5.293900442946749,
params {'n_estimators': 123, 'max_depth': 10, 'min_samples_split': 3, 'min_samples_leaf': 3}


[I 2023-08-18 20:17:20,861] Trial 0 finished with value: 22.523682791311398 and parameters: {'n_estimators': 351, 'max_depth': 4, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 22.523682791311398.
[I 2023-08-18 20:17:32,052] Trial 1 finished with value: 8.864328431288603 and parameters: {'n_estimators': 221, 'max_depth': 10, 'min_samples_split': 3, 'min_samples_leaf': 7}. Best is trial 1 with value: 8.864328431288603.
[I 2023-08-18 20:17:42,005] Trial 2 finished with value: 8.823585028824228 and parameters: {'n_estimators': 193, 'max_depth': 10, 'min_samples_split': 9, 'min_samples_leaf': 4}. Best is trial 2 with value: 8.823585028824228.
[I 2023-08-18 20:17:50,740] Trial 3 finished with value: 15.843700311817175 and parameters: {'n_estimators': 277, 'max_depth': 6, 'min_samples_split': 7, 'min_samples_leaf': 6}. Best is trial 2 with value: 8.823585028824228.
[I 2023-08-18 20:17:54,661] Trial 4 finished with value: 22.51378549478323 and parameters: {'n_esti

[I 2023-08-18 20:27:08,430] Trial 37 finished with value: 9.84905388636423 and parameters: {'n_estimators': 415, 'max_depth': 9, 'min_samples_split': 6, 'min_samples_leaf': 4}. Best is trial 24 with value: 8.819803547686748.
[I 2023-08-18 20:27:29,226] Trial 38 finished with value: 8.844682928593192 and parameters: {'n_estimators': 409, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 6}. Best is trial 24 with value: 8.819803547686748.
[I 2023-08-18 20:27:43,500] Trial 39 finished with value: 15.83757459046452 and parameters: {'n_estimators': 450, 'max_depth': 6, 'min_samples_split': 7, 'min_samples_leaf': 4}. Best is trial 24 with value: 8.819803547686748.
[I 2023-08-18 20:27:52,344] Trial 40 finished with value: 22.5200921723343 and parameters: {'n_estimators': 406, 'max_depth': 4, 'min_samples_split': 3, 'min_samples_leaf': 8}. Best is trial 24 with value: 8.819803547686748.
[I 2023-08-18 20:28:11,144] Trial 41 finished with value: 8.824388259188067 and parameters: {'n_e

Best Trial: score 8.818991916898455,
params {'n_estimators': 287, 'max_depth': 10, 'min_samples_split': 7, 'min_samples_leaf': 3}


In [26]:
print(rf_1.value)
print(rf_2.value)
print(rf_3.value)

9.791609186544664
5.293900442946749
8.818991916898455


In [27]:
rf_best_scores[0]

6.429082832136099

In [28]:
rf_hyperparams

[{'n_estimators': 460,
  'max_depth': 10,
  'min_samples_split': 6,
  'min_samples_leaf': 3},
 {'n_estimators': 173,
  'max_depth': 10,
  'min_samples_split': 10,
  'min_samples_leaf': 3},
 {'n_estimators': 123,
  'max_depth': 10,
  'min_samples_split': 3,
  'min_samples_leaf': 3},
 {'n_estimators': 287,
  'max_depth': 10,
  'min_samples_split': 7,
  'min_samples_leaf': 3}]

# SVR

In [33]:
#### svr for ensemble ####
from sklearn.svm import SVR

svr_best_scores = []
svr_hyperparams = []

## cluster 0
X_train, X_valid, y_train, y_valid = train_test_split(X_0_train, y_0_log, random_state=42)


def objective(trial):
    param = {
        'C': trial.suggest_loguniform('C', 0.3, 10),
        'degree': trial.suggest_int('degree',2,7),
        'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
        'epsilon' : trial.suggest_float('epsilon',0.05,0.5)
    }

    model = SVR(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm

study_0 = optuna.create_study(study_name = 'SVR Optuna_0')
study_0.optimize(objective, n_trials=50, timeout=24000)

svr_0 = study_0.best_trial
svr_0_param = svr_0.params
print('Best Trial: score {},\nparams {}'.format(svr_0.value, svr_0_param))

svr_best_scores.append(svr_0)
svr_hyperparams.append(svr_0_param)

## cluster 1
X_train, X_valid, y_train, y_valid = train_test_split(X_1_train, y_1_log, random_state=42)


def objective(trial):
    param = {
        'C': trial.suggest_loguniform('C', 0.3, 10),
        'degree': trial.suggest_int('degree',2,7),
        'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
        'epsilon' : trial.suggest_float('epsilon',0.05,0.5)
    }

    model = SVR(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm

study_1 = optuna.create_study(study_name = 'SVR Optuna_1')
study_1.optimize(objective, n_trials=50, timeout=24000)

svr_1 = study_1.best_trial
svr_1_param = svr_1.params
print('Best Trial: score {},\nparams {}'.format(svr_1.value, svr_1_param))

svr_best_scores.append(svr_1)
svr_hyperparams.append(svr_1_param)

## cluster 2
X_train, X_valid, y_train, y_valid = train_test_split(X_2_train, y_2_log, random_state=42)


def objective(trial):
    param = {
        'C': trial.suggest_loguniform('C', 0.3, 10),
        'degree': trial.suggest_int('degree',2,7),
        'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
        'epsilon' : trial.suggest_float('epsilon',0.05,0.5)
    }

    model = SVR(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm

study_2 = optuna.create_study(study_name = 'SVR Optuna_2')
study_2.optimize(objective, n_trials=50, timeout=24000)

svr_2 = study_2.best_trial
svr_2_param = svr_2.params
print('Best Trial: score {},\nparams {}'.format(svr_2.value, svr_2_param))

svr_best_scores.append(svr_2)
svr_hyperparams.append(svr_2_param)



[I 2023-08-18 20:39:40,068] A new study created in memory with name: SVR Optuna_0
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.3, 10),
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
[I 2023-08-18 20:39:41,104] Trial 0 finished with value: 9.84109579303507 and parameters: {'C': 0.49164724051919495, 'degree': 3, 'gamma': 0.002684073021484914, 'epsilon': 0.08945245894172539}. Best is trial 0 with value: 9.84109579303507.
C:\Users\yeji4\AppData\Local\T

C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.3, 10),
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
[I 2023-08-18 20:39:46,787] Trial 10 finished with value: 8.602190747123958 and parameters: {'C': 9.525627831504174, 'degree': 6, 'gamma': 0.009862436605855417, 'epsilon': 0.12959889085244158}. Best is trial 10 with value: 8.602190747123958.
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:13: FutureWarning: suggest_loguniform has been d

[I 2023-08-18 20:39:56,659] Trial 19 finished with value: 9.637341605727865 and parameters: {'C': 7.847573183911307, 'degree': 7, 'gamma': 0.005606060654938086, 'epsilon': 0.2213399490343299}. Best is trial 11 with value: 8.06384956314882.
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.3, 10),
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
[I 2023-08-18 20:39:57,171] Trial 20 finished with value: 9.073058616403443 and parameters: {'C': 4.378050226721

C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.3, 10),
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
[I 2023-08-18 20:40:07,429] Trial 29 finished with value: 9.202386806034038 and parameters: {'C': 5.4383594697883755, 'degree': 4, 'gamma': 0.0029702697588292015, 'epsilon': 0.09779689625130894}. Best is trial 11 with value: 8.06384956314882.
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:13: FutureWarning: suggest_loguniform has been 

[I 2023-08-18 20:40:18,584] Trial 38 finished with value: 8.800629975735355 and parameters: {'C': 6.861600776169344, 'degree': 7, 'gamma': 0.0037221348522863255, 'epsilon': 0.05162630344987404}. Best is trial 11 with value: 8.06384956314882.
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.3, 10),
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
[I 2023-08-18 20:40:19,500] Trial 39 finished with value: 9.082785425007193 and parameters: {'C': 4.8453804159

...

[I 2023-08-19 03:25:39,054] Trial 38 finished with value: 11.582708510515747 and parameters: {'C': 3.463690913735619, 'degree': 5, 'gamma': 0.003713399109752947, 'epsilon': 0.12616591647707517}. Best is trial 15 with value: 7.120443299068501.
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:75: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.3, 10),
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:77: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
[I 2023-08-19 03:25:47,343] Trial 39 finished with value: 9.85165992467351 and parameters: {'C': 3.9059208954

C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:75: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.3, 10),
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:77: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
[I 2023-08-19 03:26:45,750] Trial 48 finished with value: 6.9470096302853594 and parameters: {'C': 4.338666752544177, 'degree': 7, 'gamma': 0.008751689212988465, 'epsilon': 0.05043449298864647}. Best is trial 43 with value: 6.91983077266165.
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:75: FutureWarning: suggest_loguniform has been d

Best Trial: score 6.91983077266165,
params {'C': 8.10435868807086, 'degree': 6, 'gamma': 0.007308027315013699, 'epsilon': 0.05371389234112106}


[I 2023-08-19 03:27:30,838] Trial 0 finished with value: 19.63307745201347 and parameters: {'C': 1.9755320678459631, 'degree': 6, 'gamma': 0.0011316479088138475, 'epsilon': 0.1842978704223157}. Best is trial 0 with value: 19.63307745201347.
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:106: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.3, 10),
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/916625544.py:108: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
[W 2023-08-19 03:27:59,316] Trial 1 failed with parameters: {'C': 7.101804938454319, 'degree': 3, 'gamma': 0.

KeyboardInterrupt: 

In [36]:
## cluster 3
X_train, X_valid, y_train, y_valid = train_test_split(X_3_train, y_3_log, random_state=42)


def objective(trial):
    param = {
        'C': trial.suggest_loguniform('C', 0.3, 10),
        'degree': trial.suggest_int('degree',2,7),
        'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
        'epsilon' : trial.suggest_float('epsilon',0.05,0.5)
    }

    model = SVR(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm

study_3 = optuna.create_study(study_name = 'SVR Optuna_3')
study_3.optimize(objective, n_trials=50, timeout=24000)

svr_3 = study_3.best_trial
svr_3_param = svr_3.params

print('Best Trial: score {},\nparams {}'.format(svr_3.value, svr_3_param))

svr_best_scores.append(svr_3)
svr_hyperparams.append(svr_3_param)

[I 2023-08-19 03:28:44,760] A new study created in memory with name: SVR Optuna_3
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/740552821.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.3, 10),
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/740552821.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
[I 2023-08-19 03:29:57,080] Trial 0 finished with value: 19.39344298704222 and parameters: {'C': 0.6505933936377659, 'degree': 7, 'gamma': 0.0014265963289048999, 'epsilon': 0.050777415624079}. Best is trial 0 with value: 19.39344298704222.
C:\Users\yeji4\AppData\Local\Tem

[I 2023-08-19 03:32:56,687] Trial 9 finished with value: 13.976726734034417 and parameters: {'C': 2.9350189631745844, 'degree': 5, 'gamma': 0.005249803932587776, 'epsilon': 0.17425929275639357}. Best is trial 9 with value: 13.976726734034417.
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/740552821.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.3, 10),
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/740552821.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
[I 2023-08-19 03:33:06,287] Trial 10 finished with value: 16.60156845972719 and parameters: {'C': 1.89579838253

C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/740552821.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.3, 10),
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/740552821.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
[I 2023-08-19 03:38:23,322] Trial 19 finished with value: 10.989375048677916 and parameters: {'C': 1.7829370903031851, 'degree': 7, 'gamma': 0.009996506848881375, 'epsilon': 0.07422386720661114}. Best is trial 17 with value: 10.586541228650761.
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/740552821.py:7: FutureWarning: suggest_loguniform has been d

[I 2023-08-19 03:45:35,046] Trial 28 finished with value: 11.290883517861122 and parameters: {'C': 2.180628348081603, 'degree': 6, 'gamma': 0.008820320022708886, 'epsilon': 0.09207275981377}. Best is trial 22 with value: 10.402272282681515.
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/740552821.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.3, 10),
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/740552821.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
[I 2023-08-19 03:46:34,284] Trial 29 finished with value: 10.43633064188168 and parameters: {'C': 8.0056104973910

C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/740552821.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.3, 10),
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/740552821.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
[I 2023-08-19 03:52:21,819] Trial 38 finished with value: 12.478413955072172 and parameters: {'C': 3.5238656563019815, 'degree': 7, 'gamma': 0.006893167107996649, 'epsilon': 0.14959227793272523}. Best is trial 31 with value: 10.269556685625881.
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/740552821.py:7: FutureWarning: suggest_loguniform has been d

[I 2023-08-19 03:58:37,811] Trial 47 finished with value: 10.893755513876416 and parameters: {'C': 9.670894350789105, 'degree': 4, 'gamma': 0.009221711730858935, 'epsilon': 0.1306789602943618}. Best is trial 31 with value: 10.269556685625881.
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/740552821.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.3, 10),
C:\Users\yeji4\AppData\Local\Temp/ipykernel_5168/740552821.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma' : trial.suggest_loguniform('gamma', 1e-3, 0.01),
[I 2023-08-19 03:58:53,671] Trial 48 finished with value: 13.406551373016926 and parameters: {'C': 3.5853848403

Best Trial: score 10.269556685625881,
params {'C': 6.074629578088259, 'degree': 6, 'gamma': 0.009005215359248912, 'epsilon': 0.05425167125430294}


In [37]:
print(svr_0.value)
print(svr_1.value)
print(svr_2.value)
print(svr_3.value)

8.06384956314882
12.072459597998462
6.91983077266165
10.269556685625881


In [42]:
svr_best_scores[0]

FrozenTrial(number=11, state=TrialState.COMPLETE, values=[8.06384956314882], datetime_start=datetime.datetime(2023, 8, 18, 20, 39, 46, 787838), datetime_complete=datetime.datetime(2023, 8, 18, 20, 39, 49, 57739), params={'C': 9.841530722784379, 'degree': 6, 'gamma': 0.009891573686972346, 'epsilon': 0.05090815139828966}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'C': FloatDistribution(high=10.0, log=True, low=0.3, step=None), 'degree': IntDistribution(high=7, log=False, low=2, step=1), 'gamma': FloatDistribution(high=0.01, log=True, low=0.001, step=None), 'epsilon': FloatDistribution(high=0.5, log=False, low=0.05, step=None)}, trial_id=11, value=None)

In [39]:
svr_hyperparams

[{'C': 9.841530722784379,
  'degree': 6,
  'gamma': 0.009891573686972346,
  'epsilon': 0.05090815139828966},
 {'C': 9.243789965581446,
  'degree': 3,
  'gamma': 0.009764835275967724,
  'epsilon': 0.0746547519890558},
 {'C': 8.10435868807086,
  'degree': 6,
  'gamma': 0.007308027315013699,
  'epsilon': 0.05371389234112106},
 {'C': 6.074629578088259,
  'degree': 6,
  'gamma': 0.009005215359248912,
  'epsilon': 0.05425167125430294}]

# ET

## ET : cluster 0

In [12]:
#### et for ensemble ####
import optuna
from sklearn.ensemble import ExtraTreesRegressor

et_best_scores = []
et_hyperparams = []

## cluster 0
X_train, X_valid, y_train, y_valid = train_test_split(X_0_train, y_0_log, random_state=42)

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 800),
        'criterion': 'squared_error',
        'max_depth' : trial.suggest_int('max_depth', 15, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 10)
    }
    
    model = ExtraTreesRegressor(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm

study_0 = optuna.create_study(study_name = 'ET Optuna_0')
study_0.optimize(objective, n_trials=50, timeout=24000)

et_0 = study_0.best_trial
et_0_param = et_0.params
print('Best Trial: score {},\nparams {}'.format(et_0.value, et_0_param))

et_best_scores.append(et_0.value)
et_hyperparams.append(et_0_param)

[I 2023-08-20 17:58:11,038] A new study created in memory with name: ET Optuna_0
[I 2023-08-20 17:58:11,685] Trial 0 finished with value: 6.537629627609588 and parameters: {'n_estimators': 102, 'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 8}. Best is trial 0 with value: 6.537629627609588.
[I 2023-08-20 17:58:13,859] Trial 1 finished with value: 5.356419422208844 and parameters: {'n_estimators': 220, 'max_depth': 15, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 1 with value: 5.356419422208844.
[I 2023-08-20 17:58:15,908] Trial 2 finished with value: 6.166329091843743 and parameters: {'n_estimators': 300, 'max_depth': 17, 'min_samples_split': 2, 'min_samples_leaf': 6}. Best is trial 1 with value: 5.356419422208844.
[I 2023-08-20 17:58:17,576] Trial 3 finished with value: 6.758761014751413 and parameters: {'n_estimators': 289, 'max_depth': 20, 'min_samples_split': 5, 'min_samples_leaf': 10}. Best is trial 1 with value: 5.356419422208844.
[I 2023-08-20 17:5

[I 2023-08-20 17:59:53,235] Trial 36 finished with value: 6.746641779276528 and parameters: {'n_estimators': 442, 'max_depth': 29, 'min_samples_split': 2, 'min_samples_leaf': 10}. Best is trial 35 with value: 5.092590005836303.
[I 2023-08-20 17:59:57,321] Trial 37 finished with value: 5.45298380488375 and parameters: {'n_estimators': 474, 'max_depth': 29, 'min_samples_split': 2, 'min_samples_leaf': 3}. Best is trial 35 with value: 5.092590005836303.
[I 2023-08-20 18:00:01,834] Trial 38 finished with value: 5.100930891717425 and parameters: {'n_estimators': 428, 'max_depth': 29, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 35 with value: 5.092590005836303.
[I 2023-08-20 18:00:04,403] Trial 39 finished with value: 6.630842666456537 and parameters: {'n_estimators': 434, 'max_depth': 30, 'min_samples_split': 3, 'min_samples_leaf': 9}. Best is trial 35 with value: 5.092590005836303.
[I 2023-08-20 18:00:07,972] Trial 40 finished with value: 5.467735047595145 and parameters: 

Best Trial: score 5.077049184057563,
params {'n_estimators': 389, 'max_depth': 29, 'min_samples_split': 3, 'min_samples_leaf': 2}


## ET : cluster 1

In [13]:
## cluster 1
X_train, X_valid, y_train, y_valid = train_test_split(X_1_train, y_1_log, random_state=42)

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'criterion': 'squared_error',
        'max_depth' : trial.suggest_int('max_depth', 15, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 10)
    }

    model = ExtraTreesRegressor(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm

study_1 = optuna.create_study(study_name = 'ET Optuna_1')
study_1.optimize(objective, n_trials=50, timeout=24000)

et_1 = study_1.best_trial
et_1_param = et_1.params
print('Best Trial: score {},\nparams {}'.format(et_1.value, et_1_param))

et_best_scores.append(et_1.value)
et_hyperparams.append(et_1_param)

[I 2023-08-20 18:00:44,696] A new study created in memory with name: ET Optuna_1
[I 2023-08-20 18:01:10,456] Trial 0 finished with value: 4.919862376667327 and parameters: {'n_estimators': 254, 'max_depth': 27, 'min_samples_split': 4, 'min_samples_leaf': 9}. Best is trial 0 with value: 4.919862376667327.
[I 2023-08-20 18:02:08,481] Trial 1 finished with value: 3.9550100355618074 and parameters: {'n_estimators': 468, 'max_depth': 30, 'min_samples_split': 7, 'min_samples_leaf': 2}. Best is trial 1 with value: 3.9550100355618074.
[I 2023-08-20 18:02:26,597] Trial 2 finished with value: 4.810732708729908 and parameters: {'n_estimators': 179, 'max_depth': 26, 'min_samples_split': 2, 'min_samples_leaf': 8}. Best is trial 1 with value: 3.9550100355618074.
[I 2023-08-20 18:02:41,235] Trial 3 finished with value: 4.840244352963295 and parameters: {'n_estimators': 145, 'max_depth': 22, 'min_samples_split': 8, 'min_samples_leaf': 8}. Best is trial 1 with value: 3.9550100355618074.
[I 2023-08-20 1

[I 2023-08-20 18:22:15,326] Trial 36 finished with value: 5.528166240428543 and parameters: {'n_estimators': 314, 'max_depth': 16, 'min_samples_split': 4, 'min_samples_leaf': 3}. Best is trial 33 with value: 3.8873037639932773.
[I 2023-08-20 18:22:37,833] Trial 37 finished with value: 4.926282592109417 and parameters: {'n_estimators': 220, 'max_depth': 29, 'min_samples_split': 2, 'min_samples_leaf': 9}. Best is trial 33 with value: 3.8873037639932773.
[I 2023-08-20 18:22:58,718] Trial 38 finished with value: 3.8980370405277216 and parameters: {'n_estimators': 147, 'max_depth': 26, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 33 with value: 3.8873037639932773.
[I 2023-08-20 18:23:25,908] Trial 39 finished with value: 6.100590034104228 and parameters: {'n_estimators': 276, 'max_depth': 15, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 33 with value: 3.8873037639932773.
[I 2023-08-20 18:23:52,690] Trial 40 finished with value: 4.515129088976192 and paramet

Best Trial: score 3.88349958543788,
params {'n_estimators': 262, 'max_depth': 29, 'min_samples_split': 3, 'min_samples_leaf': 2}


## ET : cluster 2

In [14]:
## cluster 2
X_train, X_valid, y_train, y_valid = train_test_split(X_2_train, y_2_log, random_state=42)

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'criterion': 'squared_error',
        'max_depth' : trial.suggest_int('max_depth', 15, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 10)
    }

    model = ExtraTreesRegressor(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm

study_2 = optuna.create_study(study_name = 'ET Optuna_2')
study_2.optimize(objective, n_trials=50, timeout=24000)

et_2 = study_2.best_trial
et_2_param = et_2.params
print('Best Trial: score {},\nparams {}'.format(et_2.value, et_2_param))

et_best_scores.append(et_2.value)
et_hyperparams.append(et_2_param)

[I 2023-08-20 18:28:19,112] A new study created in memory with name: ET Optuna_2
[I 2023-08-20 18:28:23,989] Trial 0 finished with value: 4.802063039971386 and parameters: {'n_estimators': 458, 'max_depth': 25, 'min_samples_split': 5, 'min_samples_leaf': 8}. Best is trial 0 with value: 4.802063039971386.
[I 2023-08-20 18:28:27,146] Trial 1 finished with value: 4.084879090310256 and parameters: {'n_estimators': 185, 'max_depth': 17, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 1 with value: 4.084879090310256.
[I 2023-08-20 18:28:33,286] Trial 2 finished with value: 4.108090521754935 and parameters: {'n_estimators': 426, 'max_depth': 30, 'min_samples_split': 8, 'min_samples_leaf': 2}. Best is trial 1 with value: 4.084879090310256.
[I 2023-08-20 18:28:36,840] Trial 3 finished with value: 4.715191304900716 and parameters: {'n_estimators': 324, 'max_depth': 27, 'min_samples_split': 9, 'min_samples_leaf': 7}. Best is trial 1 with value: 4.084879090310256.
[I 2023-08-20 18:28

[I 2023-08-20 18:30:22,746] Trial 36 finished with value: 4.055274984338381 and parameters: {'n_estimators': 162, 'max_depth': 24, 'min_samples_split': 7, 'min_samples_leaf': 2}. Best is trial 16 with value: 3.9625718922043434.
[I 2023-08-20 18:30:25,785] Trial 37 finished with value: 4.337680840348218 and parameters: {'n_estimators': 225, 'max_depth': 21, 'min_samples_split': 5, 'min_samples_leaf': 4}. Best is trial 16 with value: 3.9625718922043434.
[I 2023-08-20 18:30:30,558] Trial 38 finished with value: 4.184057764512513 and parameters: {'n_estimators': 319, 'max_depth': 23, 'min_samples_split': 5, 'min_samples_leaf': 3}. Best is trial 16 with value: 3.9625718922043434.
[I 2023-08-20 18:30:37,459] Trial 39 finished with value: 3.97050254430913 and parameters: {'n_estimators': 380, 'max_depth': 20, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 16 with value: 3.9625718922043434.
[I 2023-08-20 18:30:43,268] Trial 40 finished with value: 4.17506416388513 and parameters

Best Trial: score 3.9552647031128103,
params {'n_estimators': 441, 'max_depth': 22, 'min_samples_split': 2, 'min_samples_leaf': 2}


## ET : cluster 3

In [15]:
## cluster 3
X_train, X_valid, y_train, y_valid = train_test_split(X_3_train, y_3_log, random_state=42)

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'criterion': 'squared_error',
        'max_depth' : trial.suggest_int('max_depth', 15, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 10)
    }

    model = ExtraTreesRegressor(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(np.exp(np.array(y_valid)), np.exp(np.array(preds)))
    
    return sm

study_3 = optuna.create_study(study_name = 'ET Optuna_3')
study_3.optimize(objective, n_trials=50, timeout=24000)

et_3 = study_3.best_trial
et_3_param = et_3.params
print('Best Trial: score {},\nparams {}'.format(et_3.value, et_3_param))

et_best_scores.append(et_3.value)
et_hyperparams.append(et_3_param)

[I 2023-08-20 18:31:42,360] A new study created in memory with name: ET Optuna_3
[I 2023-08-20 18:31:49,289] Trial 0 finished with value: 7.655142558092027 and parameters: {'n_estimators': 271, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 10}. Best is trial 0 with value: 7.655142558092027.
[I 2023-08-20 18:31:55,805] Trial 1 finished with value: 7.258427528538033 and parameters: {'n_estimators': 216, 'max_depth': 22, 'min_samples_split': 10, 'min_samples_leaf': 5}. Best is trial 1 with value: 7.258427528538033.
[I 2023-08-20 18:32:07,613] Trial 2 finished with value: 7.588414891769372 and parameters: {'n_estimators': 453, 'max_depth': 27, 'min_samples_split': 7, 'min_samples_leaf': 9}. Best is trial 1 with value: 7.258427528538033.
[I 2023-08-20 18:32:15,291] Trial 3 finished with value: 7.138393426965045 and parameters: {'n_estimators': 247, 'max_depth': 22, 'min_samples_split': 9, 'min_samples_leaf': 4}. Best is trial 3 with value: 7.138393426965045.
[I 2023-08-20 18:

[I 2023-08-20 18:39:17,730] Trial 36 finished with value: 6.914472648203334 and parameters: {'n_estimators': 341, 'max_depth': 27, 'min_samples_split': 7, 'min_samples_leaf': 2}. Best is trial 21 with value: 6.834284465297256.
[I 2023-08-20 18:39:26,371] Trial 37 finished with value: 7.265602291020488 and parameters: {'n_estimators': 287, 'max_depth': 29, 'min_samples_split': 3, 'min_samples_leaf': 5}. Best is trial 21 with value: 6.834284465297256.
[I 2023-08-20 18:39:36,298] Trial 38 finished with value: 7.152221835861251 and parameters: {'n_estimators': 310, 'max_depth': 21, 'min_samples_split': 4, 'min_samples_leaf': 4}. Best is trial 21 with value: 6.834284465297256.
[I 2023-08-20 18:39:41,636] Trial 39 finished with value: 7.532674802543624 and parameters: {'n_estimators': 198, 'max_depth': 29, 'min_samples_split': 5, 'min_samples_leaf': 8}. Best is trial 21 with value: 6.834284465297256.
[I 2023-08-20 18:39:56,960] Trial 40 finished with value: 6.8427958276145855 and parameters:

Best Trial: score 6.834284465297256,
params {'n_estimators': 377, 'max_depth': 24, 'min_samples_split': 2, 'min_samples_leaf': 2}


In [ ]:
print(et_0.value)
print(et_1.value)
print(et_2.value)
print(et_3.value)

In [ ]:
et_best_scores[0]

In [ ]:
et_hyperparams